<a href="https://colab.research.google.com/github/taesangeom/SK-T-Academy/blob/main/DL/InceptionNet/2_InceptionNet_20241021ex_for_std.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# InceptionNet 관련 문헌

In [ ]:
# 2014년도 이미지 분류대회에서 우승을 한 모델!!!
# 1등 : 구글 인셋셥넷/구글넷
# 2등 : VGG
# ===> 우리가 1등을 했는데,,,정작 사람들은 vgg를 더 많이 사용을 했음!!!!
# 구현하기가 직관적으로 용이하고, 이해도 쉬움!!!!!
# 엇;;; 계속 버전업을 시킨 것들이 inceptionNet version up

In [ ]:
# VGG : Conv-Pool 기본 구조
#      ==> 큰 kernel의 사이즈에 대해서 연산의 효율성을 위해서 작은 Conv을 여러번 사용하겠다!!
#      ==> 적층을 해서,,,쌓아서 deep하게 구조를 해보자!!!!
# GoolgeNet : Conv-Pool 기본 구조!!!
#      ==> 좋은 재료가 있어야 좋은 결과가 나오지 않을까!!!!!
#      ==> 기존 CNN/ VGG는 동일한 크기의 kernel에 대해서 단순히 숫자만 조절해서,,,FM 추출!!!
#      ==> 다양한 재료들을 (kernel의 다양한 사이즈) 종합적인 재료를 바탕으로 처리하자!!!!!
#          + deep
#          : 어떻게 하면 연산의 효율성을 올리고, 모델의 크기를 너무 크지 않도록 할까!!!!!!
#          (vgg는 작은 conv 여러번 사용하는 것으로 모델 크기 조절)
#          (googlenet도 작은 conv을 사용해볼까....)

* 초기 버전) https://arxiv.org/pdf/1409.4842.pdf
* Inception V4) https://arxiv.org/pdf/1602.07261.pdf

- GoogleNet이라는 이름으로 초기에는 나왔음
- GoogleNet은 ILSVRC 2014 대회에서 우승을 차지한 모델
--  상위 5개 클래스 기준 오류 6.7% 달성
--  그 이훙 version 4까지 발전을 하였으며
-- `초기 googlenet을 Inception V1`이라고도 함!!

- 전형적인 Conv구조를 탈피하여 네트워크 모듈을 쌓는 방식의 `네트워크 속 네트워크`의 구조 방식으로 설계를 함
- 전체 파라미터의 수가 ILSVRC 2012에서 압도적으로 우승을 한 AlexNet대비 파라미터의 수는 10배 이상 적고, 깊은 신경망을 구성할 수 있는 효율적인 구조
- `구글넷의 네트워크 모듈을 Inception`이라고 하며, 아래 그림에서 보이는 부분을 Inception Module임! (논문 참조)



<img src="https://drive.google.com/uc?id=1i6JQXgdH2JI_grwyi6U_hX2dGt3G6_eN">

# 기본 설계 원리

* Sparsity & Density
    + Sparsity : 극히 일부 구성 요소 사이에서만 상호작용이 일어남
    + Density : 대부분의 구성 요소 사이에 상호 작용이 일어남
        + 인공신경망 관점 : 완전 연결Full Connection보다도 상관성이 있는 출력끼리 연결된  `희소 연결 구조를 갖는 편이 훨씬 효율적`일 것임
        +HW 관점 : 연산 효율의 측면에 있어서는 `조밀성을 가지는 구조가 훨씬 효율적`일 것임.
* Inception Module : 인공신경망의 성능과 컴퓨팅 자원의 효율성을 높이기 위한 `희소 연결 구조`를 가지면서, 동시에 HW적인 연산 효율을 위해서 `조밀 연산`구조를 하도록 설계가 되어 있음!



<img src="https://drive.google.com/uc?id=1i6JQXgdH2JI_grwyi6U_hX2dGt3G6_eN">

* 4가지 역할
    + 1 X 1 Conv : 채널 특징 인식
    + 3 X 3 Conv : 서로 다른 크기 특징 추출
    + 5 X 5 Conv : 서로 다른 크기 특징 추출
    + 3 X 3 MaxPool : 가장 두드러지는 특징 인식
* 대략적 컨샙 : 여러 사이즈를 가진 Conv를 통과 시켜서 나온 FeatureMap을 Concat해서 쌓아서 하자!! ==> 이 부분에서 `쌓기 위해서는 모양을 잘 맞춰`야 함!!! ( 논문에서 stride는 1로 고정하고, padding을 컨트롤 하면서 하였다고 함)

* 아래 기본적인 inception module에 대해서 가지는 문제점
    - 아래 그림과 같이 28X28X192가 들어가서 나오는 결과는 28X28X(192+128+96+256) 으로 672개의 chanel이 생성이 됨;;;;
    - 채널이 약 256에서 672개로 2.5배 정도 늘어남
    - 결론: 가장 문제점은 바로 보이는 채널의 수가 과도하게 증가하는 현상이 나타남! ==> 이는 연산량의 증가를 야기하게 됨!!!

<img src="https://drive.google.com/uc?id=1PaKN4xBknGJ9tp6gfw4xMBosj4LtS61O">

In [ ]:
# --> 아래 그림과 같은 대안을 제시를 함 : 중간 병목측 추가

* 위의 개선된 방법의 핵심은 중간에 `Bottleneck layer`를 추가를 하는 것임!!!
    - 채널의 수 감소
    - 그로 인한 연산량 감소

<img src="https://drive.google.com/uc?id=1ur1w2Y2Gy8wMYa_wAevb9mOAT6V6RIw3">

# 모델의 구조

<img src="https://drive.google.com/uc?id=10ppxqEc6PWlVqdEggbZmdfTWnuAGiKPg">

<img src="https://drive.google.com/uc?id=1PxeMPx7h6wYhj4-KdXYMZh6IMZwlZuyB">

<img src="https://drive.google.com/uc?id=1o0VD0pCJ2VT7p0gceMb2ftoHAI0IgUYl">

* 위의 그림을 보면 알 수 있듯이
    - stem : 신경망의 도입부로 Copnv/Pool로 구성이 됨 --> 초기에는 인셉션의 효과가 없어서 일반적인 Conv로 구성
    - Body : inception module 9개 쌓음( 중간 그림의 3(a) ~ 5(b)까지, 1/3/8번째에는 inception 전에 maxPool사용해서 FeatureMap의 크기를 줄임
    - Final Clf : 완전 연결 FC을 사용하는 것 대신에 AvgPool을 사용해서 파라미터의 수 줄임
    - 보조 분류기 auxliary Clf : 2개의 보조 분류기를 사용 --> 하위 계층에 경사도를 원할히 공급 & 정규화 역할을 함.( 훈련에서만 사용하고, Test에서는 사용하지 않음!!)

* 보조 분류기 : 깊어지면서 발생을 하는 Gradient Vanishing 문제를 해결하기 위해서 중간에서도 BackPropagation을 시도하기 위해서 한 계층임.
* 참고 : InceptionNet V2이후 부터는 별로 큰 효용이 없어서 1개 줄이다가 V4에서는 사용하지 않음!

In [ ]:
# 아래는 그 당시 ver1의 성능표(논문)

<img src="https://drive.google.com/uc?id=1F5v9ctAanT21YcsMwuKbKdwchYsr6RHH">

* 위의 표들은 그 전까지의 대회에서 수상한 모델들의 성능을 중심으로 표현하고 있는 것
* 7개의 모델( 앞의 모델과 동일한 사이즈 6개 + 1개 큰 모델)을 앙상블을 함
* 여러가지 resize를 하고, crop을 해서 진행을 하였다고 함 ==> 제일 best는 1장의 사진을 144개의 이미지로 crop/resize를 통해서 7개의 모델을 통해서 앙상블한 것이 제일 Best!!!!
*

In [ ]:
# 정리 : googleNet의 초기 버전의 핵심 아이디어
#       ==> FM에 다양한 크기의 Feature를 channel로 재료로 두자!!!!!
#           + deep 하게 하면,, channel이 늘어나게 되는 구조를 쓰다보니
#             (3*3, 5*5, 1*1, pool) 그냥 하게 되면 연산이 너무나도 커진다!!!
#             : 입력FM의 Channel을 통제하기 위해서 1*1 conv의 갯수로 컨트롤!!!
#               ( conv의 연산량을 어느 정도 컨트롤을 하면서 모델을 스케일업이 가능함!!!)
#       ==> 기본적인 큰 틀은 W/H는 줄이고, channel은 늘리고 하는 방식은 동일하지만,,
#           다양한 kernel에 대한 특징이 모여있다!!!!!!
#           좋은 재료들을 잘 사용하면 , 성능이 좋겠다!!!!!!!
# 단, 실제 대회 우승은 여러 모델을 사용을 했고!!!!
#     다른 연구자들이 가성비가 있는 VGG쪽을 더 선호하게 되었습니다!!!
# 초기에는.......

# 결론
- 여러 사이지의 필터를 통과한 FeatureMap을 합쳐서 Channel로 사용함
- 1X1 Conv로 파라미터의 갯수를 좀 줄여서 처리함
- V1은 auxiliary clf가 있어서 Gradient Vanishing을 상쇄하려고 하였지만, 이후 모델에서는 이 부분을 줄여나감

# Inception Version 2/3

* ref) https://arxiv.org/pdf/1512.00567.pdf
* 사람들이 여전히 자신들이 1등을 하여도 VGG를 많이 사용하기에, 자신들이 모델의 파라미터도적고 한데 왜 VGG만 사용하나에 대한 복기로 시작
* 개선방법1
    - 5X5 Conv 1번 연산이 3X3 Conv 를 2번 사용한 것 대비 연산이 약 2.78배 더 많이 사용함( 25 / 9 = 2. 78 )

<img src="https://drive.google.com/uc?id=1R91kPopUInjEAQZ23dcPpp-TSDxXnxSO">

<img src="https://drive.google.com/uc?id=1vmEOmLdKqaHCrAbN6PwpS21IsJ051xtd">

* 개선방법2
    - 비대칭을 활용
    - N X N Conv에 대한 것을 연산을 줄이기 위해서 1 X N Conv + N X 1 Conv로 처리함
    -

<img src="https://drive.google.com/uc?id=1lwpZXa1aYiRJiGmGGawXciacSkScqZwY">

<img src="https://drive.google.com/uc?id=1YDpheAwHcDkRPvVabXJ-suDWDTOFFD0l">

* 개선방법3
    - `Conv + Pool 이나 Pool + Conv 나 결론적인 부분은 동일`한데,,이 것을 `어떤 순서로 할까의 문제`가 발생함 ==> 아래 그림에서 왼쪽은 Pool먼저, 오른쪽은 Conv먼저
    - 효율적인 크기 줄이기 : 아래 그림에서 너무나 Pooling을 해서 35X35X320을 17X17X320으로 하게되면,  Representational Bottleneck( 입력 정보 확 줄어버리는 것 )이 발생하게 됨!!
    - 오른쪽으로 같이 변경을 하게 되면, 정보가 줄어드는  Representational Bottleneck은 피할 수 있지만 연산량이 엄청 증가를;;;
    - 결론 : 아래 그림과 같이 stride=2로 한 결과를  concat하자로 ==> inception과 pooling을 동시에 수행!

<img src="https://drive.google.com/uc?id=14Shr8mQhgO5lIvw65b4cW4SUU1yLUp-k">

* 개선방법4
    - Auxiliary CLF 에 대해서는 Version1 대비해서 1개만 사용을 하도록 수정
    - 1개에는 BatchNormal을 적용

<img src="https://drive.google.com/uc?id=16eOG1XUeQoheoeSaY_iSKqX7gnWf0kuA" >

* 참고) 위의 사항들을 반영해서 사람들이 직접 구현을 한 것들은 이런 기본 구조에서 조금씩 다르고, 맨 아래 표시한 것을 V3이라고 함

<img src="https://drive.google.com/uc?id=1bmEAYWEePyBcQWsEoqK7NL7Q67BJvkYp">

In [ ]:
# 최종 정리
# GoogleNet의 핵심은 FM의 Channel에 대한 다양한 구성을 하자!!!!
# +++ conv + Pooling (FM의 channel의 구성을 conv과 pool같이 합치자!!!)
#     conv(다양한 특징을 위해서 1*1, 3*3, 5*5)
#     ==> 연산의 효율성을 위해서 입력의 FM의 channel을 조절 : conv 1*1 먼저 사용
#                                conv 3*3, conv5*5 ==> conv(1,n) * conv(n,1) 연산으로 효율화!!!
#                                                    [ vgg의 기본 아이디어 발전 ]
# +++ 실질적인 성능을 보여 준 것임!!!
# 최근에는 더 deep하고, 복잡한 모델로 학습을 했습니다!!!
# ==> 공개가 되어 있습니다!!!
# +++ 나의 데이터에 맞춰서 Tunning 하는 부분으로 활용을 해야 함!!!
#     구조를 차용하거나,,,,,

In [ ]:
#++ ResNet / U-Net
#++ Transformer : ViT ( 최신 사용하는 모델!! )
# ----> 본인의 관심사/ 주제에 따라서 보면 진행을 하시면 됩니다....

# 예제 적용

In [ ]:
# 견종 데이터 셋
#https://www.kaggle.com/c/dog-breed-identification

In [ ]:
# 구글드라이브에 있는  파일에 대한 id를 가지고
# colab에 다운로드 함..
# 이것을 실행하면
# colab의 content에 train.zip, test.zip
!gdown https://drive.google.com/uc?id=1szLijK-O-QT5LA9ytCPassTtxMg-uJ_y
!gdown https://drive.google.com/uc?id=1z3n7y7RQKlseNBpb4XoRLKjTQGd8pExH

Downloading...
From (original): https://drive.google.com/uc?id=1szLijK-O-QT5LA9ytCPassTtxMg-uJ_y
From (redirected): https://drive.google.com/uc?id=1szLijK-O-QT5LA9ytCPassTtxMg-uJ_y&confirm=t&uuid=8843f963-2236-4160-85c3-5071e284581d
To: /content/train.zip
100% 361M/361M [00:02<00:00, 156MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1z3n7y7RQKlseNBpb4XoRLKjTQGd8pExH
From (redirected): https://drive.google.com/uc?id=1z3n7y7RQKlseNBpb4XoRLKjTQGd8pExH&confirm=t&uuid=380de531-0cb0-4109-af97-4f258bddb6ac
To: /content/test.zip
100% 363M/363M [00:02<00:00, 152MB/s]


In [ ]:
# 샘플 submission, labels.csv 파일 다운로드
!gdown 1Iq0uGTax-qpAQbjdYrFPEJivoFLFc0bR
!gdown 1kKSzJISTf06tVccOwAC-1HEsdHzFKuwS

Downloading...
From: https://drive.google.com/uc?id=1Iq0uGTax-qpAQbjdYrFPEJivoFLFc0bR
To: /content/sample_submission.csv
100% 25.2M/25.2M [00:00<00:00, 156MB/s]
Downloading...
From: https://drive.google.com/uc?id=1kKSzJISTf06tVccOwAC-1HEsdHzFKuwS
To: /content/labels.csv
100% 482k/482k [00:00<00:00, 161MB/s]
